# Modulo 04 - Agent con Feedback con LangGraph (Guia Evolutiva)

Notebook guiada para estudiantes iniciales de AI Engineering.

## Objetivos de aprendizaje
1. Entender que hace la arquitectura y cuando usarla.
2. Ejecutar un caso base y leer su salida sin cajas negras.
3. Ejecutar un segundo caso (coqueteo) para comparar comportamiento.
4. Visualizar flujo/grafo y conectar con decisiones de ingenieria.

## Arquitectura: definicion y criterio de uso
**Definicion:** Agent con feedback agrega una fase de evaluacion para iterar hasta cumplir estandares minimos de calidad.

**Cuando usarla:**
- Cuando una respuesta mala tiene alto costo de negocio.
- Cuando puedes permitir iteraciones controladas de mejora.
- Cuando necesitas evidencias de calidad antes de responder.

**Puente desde modulo 02:**
- En modulo 02 resolvias con prompts directos.
- Aqui convertimos esa logica en sistema trazable y mantenible.

In [ ]:
# Setup minimo y robusto (sin imports relativos fragiles)
from __future__ import annotations

import inspect
from notebook_support import ensure_repo_on_path, load_repo_module, pretty, display_mermaid_from_payload, display_graph_png

ROOT = ensure_repo_on_path()
print('Repo root:', ROOT)

In [ ]:
module = load_repo_module('04_langchain_langgraph/06_agent_feedback/Notebooks/01_agent_feedback_langgraph.py', 'nb04_agent_feedback')
run_fn = module.run_agent_feedback
print('Funcion cargada:', run_fn.__name__)

## Paso 1 - Inspeccion rapida del contrato de ejecucion

In [ ]:
print('Firma:', inspect.signature(run_fn))
doc = inspect.getdoc(run_fn) or 'Sin docstring.'
print('Docstring (primeras lineas):')
print('\n'.join(doc.splitlines()[:8]))

## Paso 2 - Ejemplo base del curso

In [ ]:
result_1 = run_fn(verbose=False)
print('Keys del resultado:')
print(sorted(result_1.keys()))

In [ ]:
def primary_output(result: dict):
    for key in ('final', 'selected', 'final_answer'):
        if key in result and result[key] is not None:
            return result[key]
    if 'final_response' in result:
        return {
            'approved': result.get('approved'),
            'feedback_rounds': result.get('feedback_rounds'),
            'evaluator_note': result.get('evaluator_note'),
            'final_response': result.get('final_response'),
        }
    return result

print('Salida principal (caso base):')
print(pretty(primary_output(result_1)))

## Paso 3 - Visualizacion del grafo real LangGraph

In [ ]:
agent = result_1.get('__agent')
if agent is None:
    raise RuntimeError('No se encontro __agent en el resultado.')
display_graph_png(agent=agent, payload=result_1, mermaid_key='__graph_mermaid')

## Paso 4 - Segundo ejemplo: asistente latino experto en enamorar

In [ ]:
profile_coqueteo = {
    'tipo_persona': 'asistente latino experto en enamorar',
    'gustos': ['salsa romantica', 'cafes bohemios', 'poesia urbana', 'paseos nocturnos'],
    'estilo': 'carismatico, coqueto, respetuoso, humor sutil',
    'contexto': 'quiere abrir conversacion con autenticidad y sin frases vacias',
}
result_2 = run_fn(profile=profile_coqueteo, verbose=False)
print('Context hash coqueteo:', result_2.get('__context_hash'))
print('Salida principal (coqueteo):')
print(pretty(primary_output(result_2)))

## Paso 5 - Comparacion guiada (base vs coqueteo)

In [ ]:
out_1 = primary_output(result_1)
out_2 = primary_output(result_2)
print('Caso base:')
print(pretty(out_1))
print('\nCaso coqueteo:')
print(pretty(out_2))

print('\nReflexion rapida:')
print('- Observa si el tono y contenido se ajustan al contexto.')
print('- Observa si la arquitectura mantiene consistencia entre casos.')

## Errores comunes para principiantes
1. Cambiar prompts sin definir un criterio de evaluacion.
2. No mirar el grafo/flujo y asumir que el sistema hizo lo correcto.
3. Mezclar demasiados cambios a la vez y perder trazabilidad.
4. No probar un segundo caso para validar robustez.

## Checklist de abstraccion (deberias poder responder)
- Que nodo/paso aporta mas valor en esta arquitectura?
- Que costo agrego esta arquitectura respecto a modulo 02?
- En que escenario real de AI Engineering la aplicarias manana?
- Que metrica usaras para saber si realmente mejoro calidad?